In [2]:
import pandas as pd
!pip install lxml
import lxml

     |████████████████████████████████| 5.8MB 13.4MB/s eta 0:00:01


Get the table using pandas. We can use "Neighbourhood" pharase  to find the correct table

In [3]:
df_list = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', match='Neighbourhood')
df=df_list[0]
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


ignoring data with 'Not assigned' in Borough

In [4]:
df = df[df.Borough != 'Not assigned']
df

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


check for duplicate postacodes

In [5]:
print(df[df['Postcode'].duplicated() == True])

    Postcode           Borough          Neighbourhood
6        M6A        North York         Lawrence Manor
11       M1B       Scarborough                Malvern
15       M4B         East York          Parkview Hill
17       M5B  Downtown Toronto        Garden District
22       M9B         Etobicoke              Islington
..       ...               ...                    ...
272      M8Y         Etobicoke               Sunnylea
282      M8Z         Etobicoke              Mimico NW
283      M8Z         Etobicoke     The Queensway West
284      M8Z         Etobicoke  Royal York South West
285      M8Z         Etobicoke         South of Bloor

[107 rows x 3 columns]


apply the duplicate postalcode fix

In [6]:
df=df.groupby('Postcode').agg({'Borough' : 'first', 'Neighbourhood' : ','.join}).reset_index().reindex(columns=df.columns)
#check the results
print(df[df['Postcode'].duplicated() == True])

Empty DataFrame
Columns: [Postcode, Borough, Neighbourhood]
Index: []


check for not assigned neighbourhood

In [7]:
print(df[df.Neighbourhood == 'Not assigned'])

   Postcode       Borough Neighbourhood
85      M7A  Queen's Park  Not assigned


Assigning the same Borough values for those cells

In [8]:
df["Neighbourhood"]=df.apply(lambda x: x['Borough'] if(x['Neighbourhood']=='Not assigned') else x['Neighbourhood'],axis=1)
#check the results
print(df[df.Neighbourhood == 'Not assigned'])

Empty DataFrame
Columns: [Postcode, Borough, Neighbourhood]
Index: []


In [9]:
df.shape

(103, 3)

Mapping postal code to location

In [10]:
g_df=pd.read_csv("Geospatial_Coordinates.csv")
df = df.merge(g_df[['Latitude', 'Longitude','Postal Code']], how = 'left',
                left_on = 'Postcode', right_on = 'Postal Code').drop(columns= ['Postal Code'])
df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437


In [14]:
# df=df.reset_index()

In [15]:
df

,index,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...,...
98,98,M9N,York,Weston,43.706876,-79.518188
99,99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
101,101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437


In [ ]:
df.to_csv('ca_data.csv')